# Les Méthodes

**Attention, bien que ce chapitre traite essentiellement des méthodes en Rust, il introduit aussi des notions du langage comme l'OWNERSHIP, le trait Copy et les RÉFÉRENCES. Ces différents aspects seront détaillés plus tard dans le cours.** 

---

- Il est possible de définir des **fonctions propres aux structures et aux énumérations**, on les appelle communément des méthodes à cause de leur similitude avec celles des langages objet. Il existe 3 types de méthodes :
> - Les méthodes **non-mutables** qui ne modifient pas les données.
> - Les méthodes **mutables** qui modifient les données.
> - Et les **constructeurs**, bien que non-indispensables, ils retourneront une instance de l'objet initialisé.

**NB** *Il n'y a pas de Destructeur à proprement parler en Rust, les objets sont automatiquement détruits lorque ils ne sont plus dans le scope.*

À partir des types suivant, nous construirons différentes méthodes:

> ```
struct Virt(usize);
struct Phys(usize);
enum Address {
    Virt(Virt),
    Phys(Phys)
}
> ```

## Les constructeurs

* Un constructeur est une fonction qui retourne une instance initialisée d'un type donne. Voici un exemple de constructeur si l'on n'utilise pas la syntaxe des implémentations de méthodes :

In [3]:
{
    #[derive(Copy, Clone, Debug, Default)]
    struct Vector {
        i: f64,
        j: f64,
    }
    // Ceci n'est pas considere comme un vrai constructeur, c'est juste une fonction !
    fn new_default_vector() -> Vector {
        Vector::default() // Default trait used
    }
    let v = new_default_vector();
    dbg!(v);
}

[src/lib.rs:34] v = Vector {
    i: 0.0,
    j: 0.0,
}


()

> Dans le code ci-dessus, new_default_vector n'est pas a proprement parler un constructeur de Vector mais seulement une fonction qui retourne un Vector !

### Creation d'un vrai constructeur

- Afin de pouvoir preparer l'implementation de methodes pour un type, on utilisera la syntaxe `impl` `MonType` `{ methods list... }` soit **impl Vector { ... }** pour notre exemple précèdent:

In [2]:
{
    #[derive(Copy, Clone, Debug, Default)]
    struct Vector {
        i: f64,
        j: f64,
    }
    // On implemente les methodes du type Vector.
    // A l'interieur de l'implementation, Self sera equivalent a Vector.
    impl Vector {}
}

()

- Ensuite, il suffit de mettre ce qui était auparavant une fonction à l'intérieur de l'implémentation de Vector :

In [5]:
{
    #[derive(Copy, Clone, Debug, Default)]
    struct Vector {
        i: f64,
        j: f64,
    }
    // On implemente les methodes du type Vector.
    // A l'interieur de l'implementation, Self sera equivalent a Vector.
    impl Vector {
        // Ici c'est un constructeur puisqu'il est dans l'implementation et retourne Self.
        fn new_default_vector() -> Self {
            // J'utilise l'implentation du trait Default ici, mais on pourrait aussi retourner Self { i: 0.0, j: 0.0}
            Self::default()
        }
    }
    // La fonction new_default_vector() est dorenavant une fonction associee a Vector.
    // La syntaxe des Type::fn() fait reference a ces fonctions. 
    let v = Vector::new_default_vector();
    dbg!(v);   
}

[src/lib.rs:40] v = Vector {
    i: 0.0,
    j: 0.0,
}


()

> *NB* *A l'intérieur de l'implémentation, `Self` en majuscule fait référence au type qui est implémenté. Bien qu'il soit tout à fait possible d'écrire explicitement le nom du type, il est plus commode d'utiliser Self dans un souci d'uniformisation.*

*Retenir la syntaxe avec les `::` de `Type::mon_constructeur()` pour appeler le constructeur ou tout autre méthodes.*

* Voici ce à quoi ressembleraient des constructeurs pour les 3 types donnes en introduction. On suit ces 3 règles:

> - On commence toujours par `impl MonType`.
> - Un constructeur devra retourner `Self` en majuscule, Self désigne le type de l'implémentation soit MonType ici.
> - Il n'est pas oblige que le nom de la fonction constructeur soit `new`.

In [7]:
#[derive(Debug)]
struct Virt(usize);
#[derive(Debug)]
struct Phys(usize);
#[derive(Debug)]
enum Address {
    Virt(Virt),
    Phys(Phys)
}

impl Virt {
    fn new(location: usize) -> Self {
        Self(location)
    }
}
impl Phys {
    fn new(location: usize) -> Self {
        Self(location)
    }
}
// Cette implémentation possède 2 constructeurs différents.
impl Address {
    fn new_physical_address(location: usize) -> Self {
        Self::Phys(Phys::new(location))
    }
    fn new_virtual_address(location: usize) -> Self {
        Self::Virt(Virt::new(location))
    }
}

- Nous pouvons utiliser les constructeurs ainsi : le nom de la fonction du Constructeur est associé au type de l'objet, syntaxe `Type` `::` `mon_constructeur(paramètres...)`.

In [8]:
let a: Address = Address::new_physical_address(0x1000);

dbg!(a);

[src/lib.rs:37] a = Phys(
    Phys(
        4096,
    ),
)


## Les méthodes non-mutables 

### Introduction

- Une méthode dite non-mutable est une méthode qui **ne modifie pas les données** de l'objet passé en paramètre. Prenons par exemple une fonction qui affiche simplement les données contenues : 

In [10]:
#[derive(Copy, Clone, Debug, Default)]
struct Complex {
    r: f64,
    i: f64,
}

// Il n'est pas toujours utile d'utiliser un constructeur.
let mon_nombre_complexe: Complex = Complex {
    r: 3.14,
    i: 7.32,
};

// Definition de la fonction dump_data.
fn dump_data(v: Complex) {
    dbg!(v);
}

dump_data(mon_nombre_complexe);

[src/lib.rs:29] v = Complex {
    r: 3.14,
    i: 7.32,
}


- Maintenant, remplaçons la fonction dump_data par une méthode dump_data :

In [12]:
#[derive(Copy, Clone, Debug, Default)]
struct Complex {
    r: f64,
    i: f64,
}

// Il n'est pas toujours utile d'utiliser un constructeur.
// Je construis ma variable a la main.
let mon_nombre_complexe: Complex = Complex {
    r: 3.14,
    i: 7.32,
};

impl Complex {
    // Definition de la methode dump_data
    fn dump_data(self) {
        dbg!(self);
    }
}

Complex::dump_data(mon_nombre_complexe);

[src/lib.rs:47] self = Complex {
    r: 3.14,
    i: 7.32,
}


> Ici `self` mais en minuscule cette fois fait référence non plus au type lui meme, mais a **une instance de ce type**.

* **NB** Rust propose une syntaxe pratique (syntaxic sugar) pour appeler les méthodes. Plutôt que d'appeler `MonType` `::` `ma_methode(mon_instance)`, on peut a la place écrire `mon_instance` `.` `ma_methode()` avec un point entre mon instance et ma méthode.

In [134]:
// Il n'est pas toujours utile d'utiliser un constructeur.
let mon_nombre_complexe: Complex = Complex {
    r: 1.98,
    i: 3.11,
};

// Tout simplement. 
mon_nombre_complexe.dump_data();

[src/lib.rs:45] self = Complex {
    r: 1.98,
    i: 3.11,
}


> Cette syntaxe est plus élégante.

- Il parait tout aussi aisé d'appeler la méthode plusieurs fois de suite...

In [146]:
let mon_nombre_complexe_2: Complex = Complex {
    r: 1.76,
    i: 6.12,
};

mon_nombre_complexe_2.dump_data();
mon_nombre_complexe_2.dump_data();

[src/lib.rs:45] self = Complex {
    r: 1.76,
    i: 6.12,
}
[src/lib.rs:45] self = Complex {
    r: 1.76,
    i: 6.12,
}


- **ET POURTANT...** L'enchaînement de deux appels successifs a la méthode dump_data n'est possible avec **fn dump_data(self)** que si ma structure Complex est Copy ! Essayons avec une structure qui n'est pas Copy :

In [136]:
// Ici il n'y a pas de derive Copy
#[derive(Clone, Debug, Default)]
struct MyStruct {
    r: f64,
    i: f64,
}

impl MyStruct {
    fn dump_data(self) {
        dbg!(self);
    }
}

let s: MyStruct = MyStruct {
    r: 1.98,
    i: 3.11,
};

s.dump_data();
s.dump_data();

Error: use of moved value: `s`

*En effet, ça ne fonctionne plus...*

> La raison n'est pas évidente à saisir pour les débutants en Rust puisqu'elle tient aux règles d'**ownership**.

> Lors de mon premier appel a la méthode dump_data, je lui ai passé **self** (ici l'instance **s** de mon Type **MyStruct**). Seulement cette méthode n'a rien retourné, et j'ai donc perdu mon instance (les données contenues dans **s** ont été supprimées juste après l'exécution de la méthode !). Ainsi, lors du second appel à la méthode dump_data, ma variable **s** n'existait plus ! On dit que ma variable a été consumée ou "moved",

> Dans l'exemple précèdent ce dernier, ma structure était Copy. Le compilateur faisait donc une copie implicite de la structure avant de l'envoyer a la méthode, ce n'est donc pas la structure que j'avais créer au début que j'affichais grâce a la méthode, mais sa copie !

> Il existe une solution à ce problème :
>> - Le passage a la méthode d'une **référence de self** plutôt que self lui-même.

*Nous reviendrons sur les règles d'ownership du langage plus tard dans ce cours.*

### Méthode avec référence de self

- Dans cette partie, plutôt que de passer **self** en paramètre, on passera une référence de self soit `&self` avec le petit symbole AND devant `&`. À l'instar des pointeurs en C, une référence d'une variable est son adresse mémoire, ainsi si je passe une référence de self, ce ne sera plus self qui est passé a la méthode, mais son adresse mémoire.

En langage C, utiliser les pointeurs et les références s'illustrerait par un code du genre :
```
int main(void)
    struct MyVector {
        float i;
        float j;
    };
    struct MyVector v;
    v.i = 3.2;
    v.j = 6;
    void dump_data(struct MyVector *v) {
        printf("i: %f, j: %f\n", v->i, v->j);
    }
    dump_data(&v);
}
```

**OUTPUT:**  
i: 3.200000, j: 6.000000

L'adresse mémoire de v est passée à la fonction dump_data via le symbole `&`.

- L'équivalent en Rust en utilisant une méthode de MyVector et qui permet le chaînage de méthode serait :

In [148]:
#[derive(Clone, Debug)]
struct MyVector {
    i: f64,
    j: f64,
}

let v: MyVector = MyVector {
    i: 3.2,
    j: 6.0,
};

impl MyVector {
    fn dump_data(&self) {
        dbg!(self);
    }
}

v.dump_data();
v.dump_data();

[src/lib.rs:92] self = MyVector {
    i: 3.2,
    j: 6.0,
}
[src/lib.rs:92] self = MyVector {
    i: 3.2,
    j: 6.0,
}


*Ce code fonctionne maintenant parfaitement, le passage par référence a réglé le problème !*

> Dans la plupart des cas, lorsque l'on écrit des méthodes en Rust, on passera la référence de self plutôt que le self lui-même, pour rappel self avec un s minuscule désigne une "variable", que l'on peut aussi appeler "objet", ou une "instance" d'un Type.

**NB** Bien qu'il n'y en pas eu dans les exemples précédents, tout comme les fonctions, les méthodes peuvent très bien retourner quelque chose ainsi que posséder d'autres paramètres en plus de **self**.
```
impl MyType {
  fn non_mutable_method(&self, ...some_other_parameters) -> ...some_returning_values {
      ...some code
  }
}
```

## Les méthodes mutables

- Sans surprise, une méthode mutable utilisera le mot-clef `mut` pour self soit `&mut self` :
```
impl MyType {
    fn mutable_method(&mut self, ...some_other_parameters) -> ...some_returning_values {
        ...some code
    }
}
```
- Une méthode mutable modifie l'objet :

In [149]:
#[derive(Debug)]
struct Length(usize);

impl Length {
    fn add_quantity(&mut self, q: usize) {
        self.0 += q;
    }
}
let mut l = Length(5);
dbg!(&l);
l.add_quantity(3);
l.add_quantity(8);
dbg!(&l);

[src/lib.rs:282] &l = Length(
    5,
)
[src/lib.rs:285] &l = Length(
    16,
)


**NB** La variable **l** doit ici être déclarée comme mutable puisqu'elle est envoyée à une méthode qui prend d'elle une référence mutable. La méthode ayant la possibilité de modifier la variable, ce serait un non-sens que cette dernière fut déclaré sans le mot-clef **mut**.

On note aussi que la variable **l** est aussi passée par référence a la macro dbg! Afin qu'elle ne soit pas "moved" et que l'on puisse toujours y accéder dans la suite du code.

## Exercices

### Que contient la variable 'm' à la fin du code ? Ce code est-il correct ?

In [139]:
#[derive(Copy, Clone, Debug)]
struct Mass(usize);

impl Mass {
    fn add_quantity(mut self, q: usize) {
        self.0 += q;
    }
}
let mut m = Mass(5);
dbg!(&m);
m.add_quantity(3);
m.add_quantity(8);
// dbg!(&m);

[src/lib.rs:282] &m = Mass(
    5,
)


Hélas, ce ne sont que des copies de m qui ont été envoyées a la méthode add_quantity, m vaut donc toujours 5 à la fin. Le prototype de add_quantity aurait du être avec une référence de self:
```
fn add_quantity(&mut self, q: usize)
```

### On cherche à chaîner les méthodes, quelle correction doit-on apporter au code de la méthode ?

In [13]:
#[derive(Copy, Clone, Debug)]
struct Speed(usize);

impl Speed {
    fn add_quantity(&mut self, q: usize) {
        self.0 += q;
    }
}
let mut s = Speed(5);
dbg!(&s);
s.add_quantity(3).add_quantity(8); // Chainage
dbg!(&s);

Error: no method named `add_quantity` found for unit type `()` in the current scope

Ici, le problème vient du fait que notre méthode ne retourne rien, aucun type, représenté par un tuple vide. Et la méthode add_quantity n'est pas associée a (), mais au type Speed. Afin que l'on puisse utiliser la syntaxe du chaînage dans le code, il faut que la méthode retourne une référence mutable de self, c'est-à-dire que la référence mutable de l'objet lui-même soit retournée.
```
fn add_quantity(&mut self, q: usize) -> &mut Self {
        self.0 += q;
        self
    }
```
Contrairement aux types eux-mêmes qui ne sont pas forcément Copy, les références sur ces derniers pourront toujours être copies.
```
    #[derive(Debug)]
    struct Test {
        a: usize,
        b: f64,
    }
    let test = Test { a: 43, b: 5.2 };
    let ref_1 = &test;
    let ref_2 = &test;
    let ref_3 = ref_1;
    dbg!(ref_1);
    dbg!(ref_2);
    dbg!(ref_3);
```